<a href="https://colab.research.google.com/github/Nagarjunasagar/Loan_repayment_prediction/blob/main/Loan_Repayment_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loan Repayment Prediction**

## **Introduction**
Objective of this work is to predict the Loan repayment ability of the applicant based on the historical loan application data.

## This is an standard superviced classification problem statement
- Supervised : Training data which includes labelles is used to train the model to predict labels.
- Clasiification : The label is a binary variable where, 
  - 0 Represents Repayment of Loan on time
  - 1 Represents Having difficulty in repayment of loan





# **Data**
Data is from kaggle, rovided by 'Home Credit' a service dedicated to provided lines of credit (loans) to the unbanked population, as part of 'Home Credit Default Risk Competition'.
Actual dataset on kaggle has 7 different sources of data and data in the form of .csv files. Here in this work I used only two files 'application_train.csv' and 'application_test.csv' source of these data is 'Home credit' itself(Some other data is from a Bureau, which is not included in this work). 

## Understanding data 
- Training Data: 
 - 'application_train.csv' is the training data wich has information about each  loan application at 'Home Credit', Every loan has its own row and is defined by the feature `SK_ID_CURR`. 
 - 'application_train.csv' comes with `TARGET` is a binary variable representing, 0 : Loan repaid and 1 : The loan was not paid.

- Testing data :  
 - 'application_test.csv' is the testing data, has all the features/columns in training dataset except `TARGET`
- More details about columns is provided in 'columns_description.xlsx' file
 


### Libraries
We are using typical Datascience Stack `numpy`, `pandas`, `sklearn`, `matplotlib`, `seaborn`

In [ ]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd

# matplotlib and seaborn for plotting
import seaborn as sns
import matplotlib.pyplot as plt

# To suppress warnings
import warnings
warnings.frilterwarnings('ignore')

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder





